In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

This notebook will clean up the combined public housing buildings/developments/agencies datafile so that it is ready to be merged with the FEMA NFIRS dataset in the subsequent dataset.

In [2]:
public = pd.read_csv('../data/buildings_dev_pha_2.1.csv')

C:\Users\jackv\AppData\Local\Temp\ipykernel_5024\3878585035.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  public = pd.read_csv('../data/buildings_dev_pha_2.1.csv')


In [3]:
public.sample()

,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,x,y,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_participant_code,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit
106518,106519,LA004,HOUSING AUTHORITY OF LAKE CHARLES,LA004000003,LLOYD OAKS,PROJECT HOUSING,004007,SF,INAPCP,5.983010e+09,NaN,7/31/1975 12:00:00 AM,1,1,0,0,1,0.0,0,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,22.0,Calcasieu,19.0,700.0,4.0,4003.0,NaN,NaN,Calcasieu,19.0,94786.0,District 5,41155.0,Lake Charles city (part),C1,NaN,9999.0,NaN,29340.0,"Lake 

-4 indicates the value was anonymized because the property had 10 or fewer residents.

In [4]:
public = public.replace(-4, np.nan)

In [5]:
pd.set_option('display.max_rows', 300)
display((public.isna().sum() / len(public)).sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

place_inc2kx                      1.000000
dpvrc                             1.000000
dpv                               1.000000
dpvnost                           1.000000
dpvact                            1.000000
urb_out                           0.999406
zip_class                         0.993871
necta_nm                          0.967178
annl_expns_amnt_prev_yr           0.951648
pct_disabled_ge62                 0.951528
pct_age62plus                     0.951528
pct_age51_61                      0.951528
pct_age25_50                      0.951528
pct_lt24_head                     0.951528
pct_disabled_all                  0.951528
pct_ge20k                         0.951528
pct_disabled_lt62                 0.951528
pct_female_head_child             0.951528
pct_minority                      0.951528
pct_female_head                   0.951528
pct_1adult                        0.951528
pct_2adults                       0.951528
pct_age85plus                     0.951528
months_wait

In [6]:
pd.set_option('display.max_rows', 300)
display(public.dtypes)
pd.set_option('display.max_rows', 10)

objectid                            int64
participant_code                   object
formal_participant_name            object
development_code                   object
project_name                       object
building_name                      object
building_number                    object
building_type_code                 object
building_status_type_code          object
national_bldg_id                  float64
construct_date                     object
dofa_actual_dt                     object
total_dwelling_units                int64
acc_units                           int64
total_occupied                      int64
regular_vacant                      int64
total_units                         int64
pct_occupied                      float64
number_reported                     int64
pct_reported                      float64
months_since_report               float64
pct_movein                        float64
people_per_unit                   float64
people_total                      

In [8]:
columns_to_drop = [
    'x', 
    'y', 
    'formal_participant_name', 
    'project_name',
    'building_name',
    'building_number',
    'national_bldg_id',
    'state2kx', 
    'cnty_nm2kx',
    'cnty2kx',
    'tract2kx',
    'bg2kx',
    'block2kx',
    'dpvact',
    'dpvnost', 
    'curcnty_nm', 
    'curcnty', 
    'curcosub', 
    'curcosub_nm',
    'place2kx', 
    'place_nm2kx', 
    'place_cc2kx', 
    'place_inc2kx', 
    'msa', 
    'cbsa', 
    'cbsa_nm', 
    'necta', 
    'necta_nm', 
    'zcta2kx',
    'dpbc',
    'dpbc_cksum',
    'c1pgrc',  
    'c1pdrc',
    'c1psrc',
    'c1parc',
    'c1pzrc',
    'lat',
    'lon',
    'rc2kx',
    'stm2kx',
    'fcd_fips91',
    'hlc',
    'dpv', 
    'dpvrc', 
    'std_addr',
    'urb_out', 
    'msg2kx',
    'county_level',
    'place_level',	
    'tract_level',	
    'blkgrp_level',
    'county_level',
    'place_level',
    'tract_level',
    'blkgrp_level',
    'dev_buil_nu_entrance',
    'dev_development_code',
    'pha_participant_code'
]

public = public.drop(columns=columns_to_drop)

In [9]:
public['construction_year'] = pd.to_datetime(public['construct_date']).dt.year
public['dofa_actual_year'] = pd.to_datetime(public['dofa_actual_dt']).dt.year
public = public.drop(['construct_date', 'dofa_actual_dt'], axis=1)

C:\Users\jackv\AppData\Local\Temp\ipykernel_5024\1670933827.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  public['construction_year'] = pd.to_datetime(public['construct_date']).dt.year
C:\Users\jackv\AppData\Local\Temp\ipykernel_5024\1670933827.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  public['dofa_actual_year'] = pd.to_datetime(public['dofa_actual_dt']).dt.year


In [10]:
public.building_type_code = public.building_type_code.str.upper()

In [11]:
public.sample()

,objectid,participant_code,development_code,building_type_code,building_status_type_code,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,msa_nm,metro,micro,std_city,std_st,std_zip5,std_zip9,zip_class,std_zip11,addr_type,apt_type,c1pprb,msgusps,lvl2kx,ur,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit,construction_year,dofa_actual_year
105277,105278,RQ005,RQ005009019,WU,INAPCP,8,8,8,0,8,100.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,PONCE,PR,717.0,2431.0,NaN,717243137.0,H,Apt,2.0,80% Overall probable correctness,B,U,N,,3/6/2023 8:50:52 AM,NaN,NaN,53753,414.0,243.0,3799639.0,1958214.0,11.84,3.95,93.019997,1034.0,N,M,404.0,404.0,404.0,402.0,2.0,53753.0,99.51,401.0,99.75,6.0,15.46,1.8,735.0,103.0,788.0,4449.0,2427.0,47.08,25.83,14.17,5.83,7.08,20.95,6.73,31.42,19.32,83.54,72.82,4.49,28.68,73.32,30.92,6.9,19.51,6.12,8.98,51.37,19.2,20.45,0.75,100.0,0.0,0.0,0.0,100.0,4.0,141.0,98.0,59.0,23.94,35.41,40.65,38.4,99.38,58.01,55.64,Standard Performer,"EXTRA LARGE (5,000+)",12589,"EXTRA LARGE (5,000+)","EXTRA LARGE (5,000+)",6/30,Combined,11242,294535876,246835333,177542724,0,0,0,43029.0,12.74,4.68,95.330002,2104.0,536617874.0,305669993.0,13 RES LOPEZ NUSSA,137,1960.0,1960.0


In [12]:
pd.set_option('display.max_rows', 30)
display(public.select_dtypes(include='object') \
    .nunique().sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

corrected_address                 156377
development_code                    6493
std_city                            3444
participant_code                    2754
corrected_unit                      1763
msa_nm                               306
std_st                                54
msgusps                               14
pha_ha_section_8_size_category         6
building_type_code                     6
addr_type                              6
building_status_type_code              6
pha_ha_combined_size_category          6
pha_phas_designation                   6
pha_ha_low_rent_size_category          6
apt_type                               5
lvl2kx                                 5
rad_type                               4
pha_ha_fye                             4
pha_ha_program_type                    2
rad_chap_indr                          2
dev_scattered_site_ind                 2
ur                                     2
zip_class                              2
dev_pd_status_ty

In [13]:
print([x for x in public.columns if 'zip' in x.lower()])

['std_zip5', 'std_zip9', 'zip_class', 'std_zip11']


In [14]:
public['std_zip5'] = public.std_zip5.astype(str)
public['std_zip9'] = public.std_zip9.astype(str)
public['std_zip11'] = public.std_zip11.astype(str)

In [15]:
public['encoded_development_code'] = LabelEncoder().fit_transform(public.development_code)
public['encoded_participant_code'] = LabelEncoder().fit_transform(public.participant_code)
public['encoded_msa_nm'] = LabelEncoder().fit_transform(public.msa_nm)
public['encoded_zip5'] = LabelEncoder().fit_transform(public.std_zip5)
public['encoded_zip9'] = LabelEncoder().fit_transform(public.std_zip9)
public['encoded_zip11'] = LabelEncoder().fit_transform(public.std_zip11)

public = public.drop([
    'development_code',
    'participant_code',
    'msa_nm',
    'std_zip9',
    'std_zip11'
], axis=1)

In [16]:
placeholder = "MISSING_VALUE" 

# List of columns to be one-hot encoded
columns_to_encode = [
    'msgusps',
    'pha_ha_section_8_size_category',
    'building_type_code',
    'addr_type',
    'building_status_type_code',
    'pha_ha_combined_size_category',
    'pha_phas_designation',
    'pha_ha_low_rent_size_category',
    'apt_type',
    'lvl2kx',
    'rad_type',
    'pha_ha_fye',
    'pha_ha_program_type',
    'rad_chap_indr',
    'dev_scattered_site_ind',
    'ur',
    'zip_class',
    'dev_pd_status_type_code',
    'last_updt_dttm'
]

# Fill NaN values with placeholder
for column in columns_to_encode:
    public[column] = public[column].fillna(placeholder)

# One-hot encode the columns
for column in columns_to_encode:
    dummies = pd.get_dummies(public[column], prefix=column)
    public = pd.concat([public, dummies], axis=1)
    public.drop([column], axis=1, inplace=True)

In [17]:
# Encode address info, but don't drop so we can match later
public['encoded_city'] = LabelEncoder().fit_transform(public.std_city)

public['std_st'] = public['std_st'].fillna(placeholder)
dummies = pd.get_dummies(public['std_st'], prefix='std_st')
public = pd.concat([public, dummies], axis=1)

public['encoded_zip5'] = LabelEncoder().fit_transform(public.std_zip5)

In [18]:
public.shape

(191885, 310)

In [19]:
pd.set_option('display.max_rows', 350)
display(public.dtypes.sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

std_st                                                   object
corrected_unit                                           object
corrected_address                                        object
std_zip5                                                 object
std_city                                                 object
dev_pct_5k_lt10k                                        float64
dev_people_total                                        float64
dev_rent_per_month                                      float64
dev_spending_per_month                                  float64
dev_hh_income                                           float64
dev_person_income                                       float64
dev_pct_lt5k                                            float64
dev_median_inc_amnt                                     float64
dev_pct_10k_lt15k                                       float64
dev_pct_movein                                          float64
dev_pct_15k_lt20k                       

Export the cleaned file for future analysis.

In [21]:
public.to_csv('../data/buildings_dev_pha_clean_2.1.csv', index=False)